# MLflow Example: Tracking Two Model Training Runs

In [11]:
# === Model Training and Experimentation ===

# Import necessary libraries
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_digits


In [12]:

# Load dataset
data = load_digits()
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.2, random_state=42)

# Define a function to train a model and return its accuracy
def train_model(n_estimators, max_depth):
    # Train a RandomForestClassifier
    model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
    model.fit(X_train, y_train)

    # Make predictions
    predictions = model.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)

    print(f"Model with n_estimators={n_estimators}, max_depth={max_depth} achieved accuracy={accuracy:.4f}")
    return model, accuracy

In [13]:

# Train models with different hyperparameter configurations
print("Training Model 1...")
model_1, acc_1 = train_model(n_estimators=20, max_depth=5)

Training Model 1...
Model with n_estimators=20, max_depth=5 achieved accuracy=0.9389


In [14]:
print("\nTraining Model 2...")
model_2, acc_2 = train_model(n_estimators=100, max_depth=10)



Training Model 2...
Model with n_estimators=100, max_depth=10 achieved accuracy=0.9722


Now adding MLFlow

In [15]:
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_digits

In [16]:
# get this code from dagshub
import dagshub
dagshub.init(repo_owner='Bardakor', repo_name='mlflow-with-daghub', mlflow=True)

Initialized MLflow to track repo "Bardakor/mlflow-with-daghub"

Repository Bardakor/mlflow-with-daghub initialized!

In [17]:
data = load_digits()
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.2, random_state=42)

In [18]:
# Set MLflow experiment name. This will get created if it doesn't exist
experiment_name = "RandomForestExperiment_2"
mlflow.set_experiment(experiment_name)


<Experiment: artifact_location='mlflow-artifacts:/c25339129e814bb6abcea95b7621542a', creation_time=1751201371903, experiment_id='1', last_update_time=1751201371903, lifecycle_stage='active', name='RandomForestExperiment_2', tags={}>

In [19]:
# In cell 9, replace the train_and_log_model function with:
def train_and_log_model(n_estimators, max_depth):
    with mlflow.start_run():
        # Train a RandomForestClassifier
        model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
        model.fit(X_train, y_train)

        # Make predictions
        predictions = model.predict(X_test)
        accuracy = accuracy_score(y_test, predictions)

        # Log parameters and metrics (skip model logging due to DagHub compatibility)
        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)
        mlflow.log_metric("accuracy", accuracy)
        
        # Save model locally for Docker
        import os
        import joblib
        os.makedirs("models", exist_ok=True)
        joblib.dump(model, f"models/model_estimators_{n_estimators}_depth_{max_depth}.pkl")

        print(f"Logged RandomForest model with n_estimators={n_estimators}, max_depth={max_depth}, accuracy={accuracy:.4f}")
        return model, accuracy

In [20]:
# Train and log Model 1
print("Training and Logging Model 1...")
train_and_log_model(n_estimators=20, max_depth=5)

Training and Logging Model 1...
Logged RandomForest model with n_estimators=20, max_depth=5, accuracy=0.9389
🏃 View run thoughtful-ram-181 at: https://dagshub.com/Bardakor/mlflow-with-daghub.mlflow/#/experiments/1/runs/deb5fd8a27474b979b375fe27135aeaf
🧪 View experiment at: https://dagshub.com/Bardakor/mlflow-with-daghub.mlflow/#/experiments/1


(RandomForestClassifier(max_depth=5, n_estimators=20, random_state=42),
 0.9388888888888889)

In [21]:
# Train and log Model 2
print("\nTraining and Logging Model 2...")
train_and_log_model(n_estimators=100, max_depth=10)


Training and Logging Model 2...
Logged RandomForest model with n_estimators=100, max_depth=10, accuracy=0.9722
🏃 View run exultant-fly-426 at: https://dagshub.com/Bardakor/mlflow-with-daghub.mlflow/#/experiments/1/runs/2f585556cf30435cb095c9dcacbdbbff
🧪 View experiment at: https://dagshub.com/Bardakor/mlflow-with-daghub.mlflow/#/experiments/1


(RandomForestClassifier(max_depth=10, random_state=42), 0.9722222222222222)

In [22]:
# Instructions to visualize results
print("\nTo view the results, run the following command in your terminal:")
print("mlflow ui")
print("Then navigate to http://127.0.0.1:5000 to explore the experiment results.")


To view the results, run the following command in your terminal:
mlflow ui
Then navigate to http://127.0.0.1:5000 to explore the experiment results.


In [23]:
# New cell - Save best model for Docker
import joblib
import os

# Determine which model performed better and save as best_model.pkl
# You'll need to capture the models and accuracies from the previous cells
os.makedirs("models", exist_ok=True)

# Assuming model from cell with n_estimators=100, max_depth=10 performs better
# Load the better performing model and save as best_model.pkl
best_model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
best_model.fit(X_train, y_train)
joblib.dump(best_model, "models/best_model.pkl")

print("Best model saved for Docker deployment!")

Best model saved for Docker deployment!
